In [7]:
import pandas as pd
import pickle

In [8]:
# load dataframe of postcodes
# satisfying the distance requirements
pc_file = open('pc.p', 'rb')
pc = pickle.load(pc_file)
pc_file.close()
pc.shape

(4, 4)

In [9]:
# display dataframe
pc

,latitude,longitude,EX1,TA1
postcode,,,,
EX14,50.81272,-3.18960,27.0,29.0
EX15,50.87675,-3.33907,27.0,33.0
EX16,50.92258,-3.49122,30.0,28.0
TA21,50.97673,-3.24507,33.0,15.0
